**Data Science intern at CodeClause**

**Project : Customer Segment Prediction**

**Name : S Nikitha**

In [2]:
#install kaggle
!pip install -q kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sangunikitha","key":"614d7cfc0e951fb4b79c4515963baa3f"}'}

In [ ]:
#create a kaggle folder
!mkdir ~/.kaggle

In [6]:
#copy the kaggle.json to the folder created
! cp kaggle.json ~/.kaggle/

In [7]:
#permission for the json to act
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
#downloading the required dataset
!kaggle datasets download -d kunalgupta2616/hackerearth-customer-segmentation-hackathon

  0% 0.00/838k [00:00<?, ?B/s]
100% 838k/838k [00:00<00:00, 119MB/s]


In [11]:
!unzip hackerearth-customer-segmentation-hackathon.zip

Archive:  hackerearth-customer-segmentation-hackathon.zip
  inflating: avhacklive/SampleSubmission.csv  
  inflating: avhacklive/Test.csv     
  inflating: avhacklive/Train.csv    


In [ ]:
!pip install pycaret

**Removes Warnings**

In [12]:
import warnings
warnings.filterwarnings("ignore")

**Importing Libraries**

In [44]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import category_encoders as ce
from pycaret.classification import *
from imblearn.over_sampling import SMOTE
print("Setup Complete")

Setup Complete


**Preparation of Data**

In [45]:
df_train=pd.read_csv('/content/avhacklive/Train.csv')
df_test=pd.read_csv('/content/avhacklive/Test.csv')
df_test_id=df_test['id']

In [46]:
df_train.head()

,id,customer_age,job_type,marital,education,default,balance,housing_loan,personal_loan,communication_type,day_of_month,month,last_contact_duration,num_contacts_in_campaign,days_since_prev_campaign_contact,num_contacts_prev_campaign,prev_campaign_outcome,term_deposit_subscribed
0,id_43823,28.0,management,single,tertiary,no,285.0,yes,no,unknown,26,jun,303.0,4.0,NaN,0,unknown,0
1,id_32289,34.0,blue-collar,married,secondary,no,934.0,no,yes,cellular,18,nov,143.0,2.0,132.0,1,other,0
2,id_10523,46.0,technician,married,secondary,no,656.0,no,no,cellular,5,feb,101.0,4.0,NaN,0,unknown,0
3,id_43951,34.0,services,single,secondary,no,2.0,yes,no,unknown,20,may,127.0,3.0,NaN,0,unknown,0
4,id_40992,41.0,blue-collar,married,primary,no,1352.0,yes,no,cellular,13,may,49.0,2.0,NaN,0,unknown,0


In [47]:
df_train.isnull().sum()

id                                      0
customer_age                          619
job_type                                0
marital                               150
education                               0
default                                 0
balance                               399
housing_loan                            0
personal_loan                         149
communication_type                      0
day_of_month                            0
month                                   0
last_contact_duration                 311
num_contacts_in_campaign              112
days_since_prev_campaign_contact    25831
num_contacts_prev_campaign              0
prev_campaign_outcome                   0
term_deposit_subscribed                 0
dtype: int64

In [48]:
df_train.dtypes.value_counts()

object     10
float64     5
int64       3
dtype: int64

**Replacing Null values**

In [49]:
df_train["marital"] = df_train["marital"].fillna("unknown")
df_test["marital"] = df_test["marital"].fillna("unknown")

**Filling Null values with mean**

In [50]:
ma = df_train[df_train["marital"]=="married"]["customer_age"].mean()
uma = df_train[df_train["marital"]!="married"]["customer_age"].mean()
uk = df_train[df_train["marital"]=="unknown"]["customer_age"].mean()
print(ma)
print(uma)
print(uk)

42.840312163616794
36.85531812339332
40.39041095890411


In [51]:
df_train["marital"].value_counts()

married     18945
single       8857
divorced     3695
unknown       150
Name: marital, dtype: int64

**Train file**

In [52]:
for i in range(len(df_train)):
    if np.isnan(df_train["customer_age"][i]):
        # print(df_train["customer_age"][i])
        if df_train["marital"][i] == "single":
            df_train["customer_age"][i] = round(uma)
        if df_train["marital"][i] == "married":
            df_train["customer_age"][i] = round(ma)
        if df_train["marital"][i] == "divorced":
            df_train["customer_age"][i] = round(ma)
        if df_train["marital"][i] == "unknown":
            df_train["customer_age"][i] = round(uk)

**Test file**

In [53]:
for i in range(len(df_test)):
    if np.isnan(df_test["customer_age"][i]):
        if df_test["marital"][i] == "single":
            df_test["customer_age"][i] = round(uma)
        if df_test["marital"][i] == "married":
            df_test["customer_age"][i] = round(ma)
        if df_test["marital"][i] == "divorced":
            df_test["customer_age"][i] = round(ma)
        if df_test["marital"][i] == "unknown":
            df_test["customer_age"][i] = round(uk)

**Data Cleaning**

In [54]:
def one_hot_encoding(df,col):
    one_hot_encoder=ce.OneHotEncoder(cols=col,return_df=True,use_cat_names=True)
    df_final = one_hot_encoder.fit_transform(df)
    return df_final

In [55]:
for i in df_train.columns:
    if i == "term_deposit_subscribed":
        continue
    if df_train[i].dtypes == "object" and i !="id":
        df_train[i].fillna("unknown",inplace=True)
        df_train = one_hot_encoding(df_train,i)
    elif df_train[i].isnull().sum()>0: 
        if i == "num_contacts_in_campaign":
            df_train[i].fillna(1.0,inplace=True)
        else:
            df_train[i].fillna(round(df_train[i].mean()),inplace=True)
    
    if df_test[i].dtypes == "object" and i !="id":
        df_test[i].fillna("unknown",inplace=True)
        df_test = one_hot_encoding(df_test,i)
    elif df_test[i].isnull().sum()>0: 
        if i == "num_contacts_in_campaign":
            df_test[i].fillna(1.0,inplace=True)
        else:
            df_test[i].fillna(round(df_test[i].mean()),inplace=True)

In [56]:
columns_to_drop = ["id"]
df_train = df_train.drop(columns_to_drop,axis = 1)
df_test = df_test.drop(columns_to_drop,axis = 1)
print('Done')

Done


In [57]:
df_train.isnull().sum()

customer_age                        0
job_type_management                 0
job_type_blue-collar                0
job_type_technician                 0
job_type_services                   0
job_type_retired                    0
job_type_entrepreneur               0
job_type_admin.                     0
job_type_student                    0
job_type_housemaid                  0
job_type_self-employed              0
job_type_unknown                    0
job_type_unemployed                 0
marital_single                      0
marital_married                     0
marital_divorced                    0
marital_unknown                     0
education_tertiary                  0
education_secondary                 0
education_primary                   0
education_unknown                   0
default_no                          0
default_yes                         0
balance                             0
housing_loan_yes                    0
housing_loan_no                     0
personal_loa

**Predictive Segmentation**

In [58]:
X_train = df_train.drop("term_deposit_subscribed",axis = 1)
Y_train = df_train["term_deposit_subscribed"]
smote = SMOTE()
X_train, Y_train = smote.fit_resample(X_train, Y_train)
df_train_smoted = X_train
df_train_smoted["term_deposit_subscribed"] = Y_train

In [59]:
experiment = setup(df_train_smoted,target = "term_deposit_subscribed",imputation_type="iterative")

,Description,Value
0,Session id,8506
1,Target,term_deposit_subscribed
2,Target type,Binary
3,Original data shape,"(56506, 54)"
4,Transformed data shape,"(56506, 54)"
5,Transformed train set shape,"(39554, 54)"
6,Transformed test set shape,"(16952, 54)"
7,Numeric features,53
8,Preprocess,True
9,Imputation type,iterative


In [60]:
mod = create_model("rf")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9540,0.9938,0.9444,0.9629,0.9535,0.9080,0.9082
1,0.9596,0.9949,0.9545,0.9642,0.9593,0.9191,0.9192
2,0.9545,0.9939,0.9408,0.9673,0.9539,0.9090,0.9093
3,0.9590,0.9949,0.9479,0.9695,0.9586,0.9181,0.9183
4,0.9509,0.9927,0.9393,0.9617,0.9504,0.9019,0.9021
5,0.9532,0.9932,0.9423,0.9633,0.9527,0.9064,0.9067
6,0.9489,0.9932,0.9393,0.9577,0.9484,0.8979,0.8980
7,0.9573,0.9934,0.9439,0.9699,0.9567,0.9145,0.9149
8,0.9512,0.9935,0.9393,0.9622,0.9506,0.9024,0.9027


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [61]:
df_train

,customer_age,job_type_management,job_type_blue-collar,job_type_technician,job_type_services,job_type_retired,job_type_entrepreneur,job_type_admin.,job_type_student,job_type_housemaid,...,month_dec,last_contact_duration,num_contacts_in_campaign,days_since_prev_campaign_contact,num_contacts_prev_campaign,prev_campaign_outcome_unknown,prev_campaign_outcome_other,prev_campaign_outcome_failure,prev_campaign_outcome_success,term_deposit_subscribed
0,28.0,1,0,0,0,0,0,0,0,0,...,0,303.0,4.0,225.0,0,1,0,0,0,0
1,34.0,0,1,0,0,0,0,0,0,0,...,0,143.0,2.0,132.0,1,0,1,0,0,0
2,46.0,0,0,1,0,0,0,0,0,0,...,0,101.0,4.0,225.0,0,1,0,0,0,0
3,34.0,0,0,0,1,0,0,0,0,0,...,0,127.0,3.0,225.0,0,1,0,0,0,0
4,41.0,0,1,0,0,0,0,0,0,0,...,0,49.0,2.0,225.0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31642,58.0,0,0,0,0,0,0,1,0,0,...,0,99.0,2.0,265.0,3,0,1,0,0,0
31643,51.0,1,0,0,0,0,0,0,0,0,...,0,31.0,7.0,225.0,0,1,0,0,0,0
31644,41.0,0,0,0,0,0,0,0,0,0,...,0,89.0,2.0,225.0,0,1,0,0,0,0
31645,48.0,0,0,0,1,0,0,0,0,0,...,0,216.0,6.0,225.0,0,1,0,0,0,0


In [62]:
df_test

,customer_age,job_type_retired,job_type_blue-collar,job_type_technician,job_type_housemaid,job_type_self-employed,job_type_management,job_type_admin.,job_type_unemployed,job_type_entrepreneur,...,month_oct,month_dec,last_contact_duration,num_contacts_in_campaign,days_since_prev_campaign_contact,num_contacts_prev_campaign,prev_campaign_outcome_unknown,prev_campaign_outcome_success,prev_campaign_outcome_failure,prev_campaign_outcome_other
0,55.0,1,0,0,0,0,0,0,0,0,...,0,0,90.0,2.0,224.0,0,1,0,0,0
1,24.0,0,1,0,0,0,0,0,0,0,...,0,0,63.0,2.0,224.0,0,1,0,0,0
2,46.0,0,0,1,0,0,0,0,0,0,...,0,0,208.0,1.0,224.0,0,1,0,0,0
3,56.0,0,0,0,1,0,0,0,0,0,...,0,0,34.0,1.0,224.0,0,1,0,0,0
4,62.0,1,0,0,0,0,0,0,0,0,...,0,0,127.0,1.0,188.0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13559,29.0,0,0,0,0,0,1,0,0,0,...,0,0,94.0,1.0,224.0,0,1,0,0,0
13560,43.0,0,1,0,0,0,0,0,0,0,...,0,0,155.0,1.0,224.0,0,1,0,0,0
13561,45.0,0,1,0,0,0,0,0,0,0,...,0,0,231.0,4.0,224.0,0,1,0,0,0
13562,52.0,0,0,0,0,0,0,1,0,0,...,0,0,243.0,2.0,299.0,5,0,0,1,0


In [63]:
y_pred = predict_model(mod , data = df_test)
y_pred

,customer_age,job_type_retired,job_type_blue-collar,job_type_technician,job_type_housemaid,job_type_self-employed,job_type_management,job_type_admin.,job_type_unemployed,job_type_entrepreneur,...,last_contact_duration,num_contacts_in_campaign,days_since_prev_campaign_contact,num_contacts_prev_campaign,prev_campaign_outcome_unknown,prev_campaign_outcome_success,prev_campaign_outcome_failure,prev_campaign_outcome_other,prediction_label,prediction_score
0,55.0,1,0,0,0,0,0,0,0,0,...,90.0,2.0,224.0,0,1,0,0,0,0,0.73
1,24.0,0,1,0,0,0,0,0,0,0,...,63.0,2.0,224.0,0,1,0,0,0,0,0.86
2,46.0,0,0,1,0,0,0,0,0,0,...,208.0,1.0,224.0,0,1,0,0,0,0,0.63
3,56.0,0,0,0,1,0,0,0,0,0,...,34.0,1.0,224.0,0,1,0,0,0,0,0.77
4,62.0,1,0,0,0,0,0,0,0,0,...,127.0,1.0,188.0,1,0,1,0,0,1,0.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13559,29.0,0,0,0,0,0,1,0,0,0,...,94.0,1.0,224.0,0,1,0,0,0,0,0.70
13560,43.0,0,1,0,0,0,0,0,0,0,...,155.0,1.0,224.0,0,1,0,0,0,0,0.75
13561,45.0,0,1,0,0,0,0,0,0,0,...,231.0,4.0,224.0,0,1,0,0,0,0,0.81
13562,52.0,0,0,0,0,0,0,1,0,0,...,243.0,2.0,299.0,5,0,0,1,0,0,0.96


In [64]:
final = pd.DataFrame()
final["id"] = df_test_id
final["term_deposit_subscribed"]=round(y_pred["prediction_score"])
final.to_csv("submission_v1.csv")
final.describe()

,term_deposit_subscribed
count,13564.000000
mean,0.994397
std,0.074646
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000
